To play with the code from *vect_wavelength* branch as of 16 May 2023

In [ ]:
import sys
import importlib

import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm 

import models
import models.spotted_star

In [ ]:
importlib.reload(models)
importlib.reload(models.spotted_star)

In [ ]:
# define star model
model = models.spotted_star.StarModel(nth=1000, nr=n_annuli)

## Single Spot - single timestep

In [ ]:
# One spot

lat, lon, rspot = 25, 30, 0.2

spotted_mask, fraction_spot_out = model.lc_mask(lat, lon, rspot)

model.show(spotted_mask, axis=True)
plt.show()

plt.plot(fraction_spot_out)
plt.xlabel('R')
plt.ylabel('Mask ratio')
pass

In [ ]:
from models.spotted_star import spher_to_cart

# add planet

_, y, z = spher_to_cart(lat, lon)  # Samish center as spot
yp, zp, rp = y, z, 0.1
print(yp, zp)

#yp, zp, rp = 0.42, 0.4, 0.1

fraction_spot, fraction_planet, ff_spot, ff_planet = model.lc_mask_with_planet(spotted_mask, yp, zp, rp)

occulted_fraction = fraction_spot_out - fraction_spot.squeeze()

model.show(spotted_mask, yp, zp, rp, axis=True)
plt.axvline(0, color='black', label='meridian',
                    linestyle='dashed', linewidth=0.7)
plt.show()


_, ax = plt.subplots(4, sharex=True)
ax[0].plot(fraction_spot_out.squeeze(), label='total spot fraction')
ax[1].plot(fraction_planet.squeeze(), label='planet fraction')
ax[2].plot(fraction_spot.squeeze(), label='visible spot fraction')
ax[3].plot(occulted_fraction.squeeze(), label='occulted spot fraction') 
for x in ax:
    x.legend()
ax[3].set_xlabel('R')
plt.show()

## Full light curve

In [ ]:
import pylightcurve as plc
import astropy.units as u

star_radius = 0.8*u.solRad
planet_radius = 0.273*u.jupiterRad
planet_mass = 0.0157*u.jupiterMass
planet_period = 2 #days
planet_inc = 92 ####


ratio = (planet_radius.to(u.m)/(star_radius.to(u.m)))

sma_over_r_star = ((0.05*u.AU).to(u.m)/(star_radius).to(u.m)).base

n_hours    =  1.
time       = np.arange(0,2*n_hours*60*60, 100)

(position_x, position_y, position_z)     =  plc.planet_orbit(planet_period, sma_over_r_star, 0, planet_inc, 0, 0, 0-n_hours/24.+time/(3600.*24.))


model.show(spotted_mask, position_y, position_z, ratio, axis=True)

In [ ]:
# Taurex inputs
path = ...
temp_star    = np.load(path+'temp_star.npy') 
tr_spectrum  = np.load(path+'tm_spectrum.npy')
wavelength   = np.load(path+'wl_grid.npy')
ldc          = np.load(path+'ldc.npy')
#ldc          = np.zeros(ldc.shape)
photosphere  = np.load(path+'photosphere.npy')
spot         = np.load(path+'spot.npy')
photosphere_phoenix  = np.load(path+'photosphere_phoenix.npy')
spot_phoenix  = np.load(path+'spot_phoenix.npy')
bin_widths = np.load(path+'bin_widths.npy')
mu        = np.sqrt(1.-model.radii**2.)

def int_profile(mu,ldc):
    return 1.-ldc[:,0,None]*(1.-mu**(1./2.))-ldc[:,1,None]*(1.-mu)-ldc[:,2,None]*(1.-mu**(3./2.))-ldc[:,3,None]*(1.-mu**2.)
def int_abs(flux,ldc):
    int_zero = flux/(2.*np.pi*(1./2.-ldc[:,0]/10.-ldc[:,1]/6.-3.*ldc[:,2]/14.-ldc[:,3]/4.))
    return int_zero[:,None]*int_profile(mu,ldc)
abs_flux_phot = int_abs(photosphere_phoenix,ldc)
abs_flux_spot = int_abs(spot_phoenix,ldc)

flux_drop  = np.zeros([len(wavelength),len(time)]) 
for i in range(len(wavelength)):
    flux_drop[i,:]  =  plc.transit(ldc[i,:], np.array([tr_spectrum[i]**(1./2.)]), planet_period, sma_over_r_star, 0, planet_inc, 0, 0, 0 -n_hours/24.+time/(3600.*24.),'claret')

spotted_profile = abs_flux_phot*(1.-fraction_spot_out)+abs_flux_spot*fraction_spot_out
flux_out  = np.sum(2.0*np.pi*model.radii*model.deltar*spotted_profile,axis=1)

# containers for LC
spotlc              = np.zeros([len(time),len(wavelength)])
spotlc_no_bumps     = np.zeros([len(time),len(wavelength)])

 

# Looping assuming that vectorised wavelength is accepted
jrecord = 42

for j in tqdm(range(len(time))):

    fraction_spot, fraction_planet, ff_spot, ff_planet = model.lc_mask_with_planet(spotted_mask, position_y[j], 
                                                                                   position_z[j], np.sqrt(tr_spectrum))
    occulted_fraction     = fraction_spot_out[:,None] - fraction_spot #frazione macchiata che il pianeta sta occultando

    if j==jrecord:
        fraction_spot1, fraction_planet1, occulted_fraction1 = fraction_spot, fraction_planet, occulted_fraction
    
    try:
        assert (occulted_fraction <= fraction_planet+1e-10).all()
    except AssertionError as e:
        print(f'time {j}, {occulted_fraction.sum()} {fraction_planet.sum()}')
        raise e

    excess                = np.sum(2.0*np.pi*model.radii[:,None]*model.deltar*abs_flux_phot.T
                                   *occulted_fraction, axis=0)/np.pi
    deficit               = np.sum(2.0*np.pi*model.radii[:,None]*model.deltar*abs_flux_spot.T
                                   *occulted_fraction, axis=0)/np.pi
    
    spotlc_no_bumps[j]  = flux_out-(1.-flux_drop[:, j])*photosphere_phoenix
    spotlc[j]           = spotlc_no_bumps[j]+excess-deficit

In [ ]:
plt.plot(time, spotlc/spotlc[0])
plt.show()
plt.plot(time, spotlc_no_bumps/spotlc_no_bumps[0])
plt.show()

_, ax = plt.subplots(4, sharex=True)
plt.title(f'Timestep {jrecord}')
ax[0].plot(fraction_spot_out.squeeze())
ax[0].set_title('total spot fraction')
ax[1].plot(fraction_planet1)
ax[1].set_title('planet fraction')
ax[2].plot(fraction_spot1)
ax[2].set_title('visible spot fraction')
ax[3].plot(occulted_fraction1)
ax[2].set_title('occulted spot fraction') 
ax[3].set_xlabel('R')
plt.tight_layout()
plt.show()